# Bulk PAGE Tutorial

This notebook demonstrates the current bulk `pyPAGE` workflow using bundled demo inputs.


## 1. Imports


In [ ]:
import numpy as np
import pandas as pd
from pypage import PAGE, ExpressionProfile, GeneSets


## 2. Load expression and gene sets

Using the DESeq2-style demo table and Hallmark GMT file.


In [ ]:
expr = pd.read_csv("../example_data/test_DESeq_logFC.txt", sep="	")
exp = ExpressionProfile(
    expr["GENE"],
    expr["log2FoldChange"],
    is_bin=False,
    n_bins=9,
)

gs = GeneSets.from_gmt("../example_data/h.all.v2026.1.Hs.symbols.gmt")
print(exp)
print(gs)


## 3. Run bulk PAGE

Use fewer permutations here to keep the notebook lightweight.


In [ ]:
np.random.seed(42)
p = PAGE(
    exp,
    gs,
    function="cmi",
    n_shuffle=200,
    alpha=0.005,
    k=20,
    filter_redundant=True,
    redundancy_ratio=5.0,
    n_jobs=1,
)
results, heatmap = p.run()
results.head()


## 4. Save and view outputs


In [ ]:
results.to_csv("bulk_results.tsv", sep="	", index=False)
if heatmap is not None:
    heatmap.save("bulk_heatmap.pdf", max_rows=50)
    heatmap.to_html("bulk_heatmap.html", max_rows=50)
    heatmap.save_matrix("bulk_results.matrix.tsv")

results.shape


## 5. Manual pathway mode

Bypasses permutation testing and redundancy filtering.


In [ ]:
example_pathways = list(gs.pathways[:2])
manual_results, manual_hm = p.run_manual(example_pathways)
manual_results


## 6. CLI equivalents

```bash
pypage -e ../example_data/test_DESeq_logFC.txt \
    --gmt ../example_data/h.all.v2026.1.Hs.symbols.gmt \
    --type continuous --cols GENE,log2FoldChange --n-bins 9 --seed 42

pypage --draw-only --outdir ../example_data/test_DESeq_logFC_cont_PAGE
```
